In [11]:
#Importing the required libraries and opening the JSON file
import json
import pandas as pd
import logging 

logging.basicConfig(level=logging.DEBUG)
logging.info("Starting script...")
try:
    with open('C:/Users/dvive/Downloads/Project/Data/config.json', "r") as json_file:
        config_file = json_file.read()
except FileNotFoundError as error:
    logging.error("File not found:", error)
except Exception as error:
    logging.error("Error opening file:", error)

#Converting the JSON file to a dictionary
try:
    config_file = json.loads(config_file) 
except json.JSONDecodeError as error:
    logging.error("Error decoding JSON:", error)
except Exception as error:
    logging.error("Error loading config file:", error)
    
#Saving the Primary Key of the JSON file
primary_key = "sql_steps"

#Printing the details of the JSON file
logging.info("Printing the details of the JSON file...")
logging.debug(f"Type of Config File : {type(config_file)}")
logging.debug(f"Keys in Config File : {config_file.keys()}")
logging.debug(f"The Values of sql_steps is a list: {isinstance(config_file['sql_steps'],list)}")
logging.debug(f"Value of Key 'sql_steps' : {config_file['sql_steps']}")

#Function to check if the key exists in the JSON file
config_file.setdefault(primary_key,"Key Not found") 
#Function to check if the datatype of the variable matches the desired datatype 
def validate_datatype(var,datatype):
    if type(var) == datatype:
        return f"Datatype {datatype} Matched"
    else:
        return f"Datatype {datatype} not Matched"
    
#Calling the function to check the datatype of the variable    
logging.info("Checking the datatype of the variable...")    
logging.debug(validate_datatype(config_file,dict))

#Function to Validate Data in the config JSON file
def check_keys(*args):
    for i in range(len(config_file[primary_key])):
        try:
            logging.info(f"Checking keys in Block {i}...")
            #Saving repeating logic to variables
            sql_num_value = config_file[primary_key][i].get(args[0])
            action_value = config_file[primary_key][i].get(args[1])
            action_parsed_value = config_file[primary_key][i].get(args[1]).split("|")[0]
            action_format_value = config_file[primary_key][i].get(args[1]).split("|")[1]
            next_sql_value = config_file[primary_key][i].get(args[2])
            print()
            #If condition to check if the keys exist in the config JSON file and converting sql_num to int
            if sql_num_value is not None and action_value is not None and next_sql_value is not None:
                logging.debug(f"All Keys Found in Block {i}")
                if isinstance(sql_num_value,int)==False:
                    logging.debug("sql_num exists: ",sql_num_value," Type: ",type(sql_num_value))
                    #Typecast sql_num_value to int and save changes to config_file
                    config_file[primary_key][i][args[0]] = int(sql_num_value)
                    sql_num_value = int(sql_num_value)
                    logging.debug("Type of sql_num typecasted to: ",type(sql_num_value))
            else:
                print("All Keys not Found in Block ",i)
                if  sql_num_value is None:
                    logging.error("sql_num not Found in Block ",i)
                if  action_value is None:
                    logging.error("action not Found in Block ",i)
                if  next_sql_value is None:
                    logging.error("next_sql not Found in Block ",i)
            #If condition to check if the action is read or write
            if action_parsed_value not in ("read", "write"):
                logging.error("Error : Invalid action-",action_parsed_value,"exists in Block ",i,". ")
            else:
                logging.debug("Valid action",action_parsed_value,"exists in Block ",i,". ")   
            #If condition to check if the action is read and the format is database or parquet
            if action_parsed_value=="read" and action_format_value not in ("database","parquet"):
                        logging.error("Error : Invalid Format-",action_format_value,"exists in Block ",i,". ")
            if action_parsed_value=="read" and action_format_value in ("database","parquet"):
                logging.debug("Valid Format",action_format_value,"exists in Block ",i,". ")
            #If condition to check if the action is write and the format is parquet
            if action_parsed_value=="write" and action_format_value not in ("parquet"):
                logging.error("Error : Invalid Format-",action_format_value,"exists in Block ",i,". ")   
            if action_parsed_value=="write" and action_format_value in ("parquet"):
                logging.debug("Valid Format",action_format_value,"exists in Block ",i,". ")
        except Exception as error:
            logging.exception(f"{error}")



#Calling the function to check the keys in the config JSON file    
check_keys("sql_num","action","next_sql")   

INFO:root:Starting script...
INFO:root:Printing the details of the JSON file...
DEBUG:root:Type of Config File : <class 'dict'>
DEBUG:root:Keys in Config File : dict_keys(['sql_steps'])
DEBUG:root:The Values of sql_steps is a list: True
DEBUG:root:Value of Key 'sql_steps' : [{'sql_num': '1', 'action': 'read|publish', 'sql': "select original_claim_num_cd, parent_claim_num_cd, child_claim_num_cd, claim_ordinal_value_num from ods_vw_service.mv_consume_claim_ln_reprocess_assn where claim_ln_reprocess_tp_cd = 'RORGCL' union select original_claim_num_cd, parent_claim_num_cd, child_claim_num_cd, claim_ordinal_value_num from ods_vw_service.mv_consume_claim_ln_reprocess_assn where claim_ln_reprocess_tp_cd != 'RORGCL' and parent_claim_num_cd != child_claim_num_cd", 'location': 'Data\\Config.json', 'temp_view': 'consume_claim_ln_reprocess_assn', 'next_sql': '2'}, {'sql_num': '2', 'action': 'write|parquet', 'sql': 'select * from consume_claim_ln_reprocess_assn', 'location': 'sandbox/claims_reproce

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\dvive\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dvive\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\dvive\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dvive\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\dvive\AppData\Roaming\Python\Python311\site-packages

In [12]:
import csv
import logging

def read_csv_file(location):
    data = read_csv_file(location)
    return data

def write_csv_file(location, data):
    data.to_csv(location, index=False)


def read_parquet_file(location):
    return pd.read_parquet(location)

import fastparquet as fp
def write_parquet_file(location, data):
    fp.write(location, data)


import mysql.connector
def read_database(location):
    mydb = mysql.connector.connect(
        host="localhost",
        user="vivek",
        password="xyz",
        database="db_name"
    )
    mycursor = mydb.cursor()
    mycursor.execute("SELECT * FROM db_name")
    data = [dict(zip([i[0] for i in mycursor.description], row)) for row in mycursor.fetchall()]
    mydb.close()
    return data


def write_database(location, data):
    mydb = mysql.connector.connect(
        host="localhost",
        user="vivek",
        password="xyz",
        database="db_name"
    )
    mycursor = mydb.cursor()
    mycursor.execute("CREATE TABLE IF NOT EXISTS db_name (id INT AUTO_INCREMENT PRIMARY KEY, column1 VARCHAR(255), column2 VARCHAR(255), column3 VARCHAR(255))")
    for row in data:
        sql = "INSERT INTO db_name (column1, column2, column3) VALUES (%s, %s, %s)"
        val = (row['column1'], row['column2'], row['column3'])
        mycursor.execute(sql, val)
    mydb.commit()
    mydb.close()



def read_file(action, format, location):
    logging.basicConfig(filename='file_operations.log', level=logging.INFO)
    logging.info(f"Action: {action}, Format: {format}, Location: {location}")
    if action=='read':
        if format=='csv':
            read_csv_file(location)
        if format=='parquet': 
            read_parquet_file(location)
        if format=='database':
            read_database(location)
        else:
            raise ValueError(f"Unsupported format: {format}")
        logging.info(f"Successfully read data from {location} in {format} format")

    if action=='write':
        if format=='csv':
            write_csv_file(location, read_csv_file(location))
        if format=='parquet': 
            write_parquet_file(location, read_parquet_file(location))
        if format=='database':
            write_database(location, read_database(location))
        else:
            raise ValueError(f"Unsupported format: {format}")
        logging.info(f"Successfully wrote data to {location} in {format} format")
    else:
        raise ValueError(f"Unsupported action: {action}")

In [13]:
for i in config_file["sql_steps"]:
    if i["sql_num"]==1:
        read_file(i["action"].split("|")[0],i["action"].split("|")[1], i["location"])

INFO:root:Action: read, Format: publish, Location: Data\Config.json


ValueError: Unsupported format: publish